In [1]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
EMAIL = "lamtt284@gmail.com"
PASSWORD = "Giadinh1"

SEARCH_KEYWORD = "Data Engineer"
LOCATION = "Vietnam"


job_list_data = []

driver = webdriver.Chrome()
driver.maximize_window()

In [3]:
def login_linkedin():
    wait = WebDriverWait(driver, 20)
    temp = driver.get('https://www.linkedin.com/login')

    email_input = wait.until(EC.presence_of_element_located((By.ID, "username")))
    password_input = driver.find_element(By.ID, "password")

    email_input.send_keys(EMAIL)
    password_input.send_keys(PASSWORD)
    password_input.submit()

In [4]:

def scrape_jobs_on_page():
    
    wait = WebDriverWait(driver, 60)
    job_cards = driver.find_elements(By.CSS_SELECTOR, "a[class*='job-card-container__link']")
    time.sleep(5)
    
    for job_card in job_cards:
        
        driver.execute_script("arguments[0].click();", job_card)
        #job_card.click()
        
        ### job_title
        job_title_element = wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "a[class='ember-view']")))

        job_title = job_title_element.text
        print(job_title)

        # company
        company_element = driver.find_element(By.CSS_SELECTOR, ".job-details-jobs-unified-top-card__company-name")
        company = company_element.text
        print(company)

        
        ### job_detail
        job_detail_element = driver.find_element(By.CSS_SELECTOR, ".jobs-description-content__text--stretch")
        job_detail = job_detail_element.text
        #print(job_detail)

        ### location
        location_element = driver.find_element(By.CSS_SELECTOR,".job-card-container__metadata-wrapper")
        location = location_element.text
        print(location)

        job_list_data.append({
            "Title": job_title,
            "Company": company,
            "Location": location,
            "Description": job_detail,
        })

        print(job_list_data)
        
    return job_list_data

In [5]:
def find_and_click_next_page():
    try:
        # Tìm nút "Next" hoặc nút số trang tiếp theo
        next_button = driver.find_element(By.CSS_SELECTOR, ".jobs-search-pagination__button--next")
        next_button.click()
        return True
    
    except (NoSuchElementException, ElementNotInteractableException):
        # Không tìm thấy nút "Next" hoặc không thể tương tác, dừng vòng lặp
        return False

In [ ]:
def main():
    login_linkedin()

    # Lần đầu tiên truy cập trang tìm kiếm
    # Tạo URL tìm kiếm
    time.sleep(5)
    print("Searching")

    search_url = f"https://www.linkedin.com/jobs/search/?keywords={SEARCH_KEYWORD.replace(' ', '%20')}&location={LOCATION.replace(' ', '%20')}"
    driver.get(search_url)
    time.sleep(5)
    
    job_list_data = []
    
    page_number = 1
    while True:
        print(f"--- Đang lấy dữ liệu từ trang {page_number} ---")
        
        # Lấy dữ liệu của tất cả các job trên trang hiện tại
        jobs_on_current_page = scrape_jobs_on_page()
        job_list_data.extend(jobs_on_current_page)
        
        # Chuyển trang
        can_continue = find_and_click_next_page()
        if not can_continue:
            print("Đã đến trang cuối cùng. Dừng lại.")
            break
        
        # Chờ trang mới tải xong trước khi tiếp tục vòng lặp
        time.sleep(3) 
        page_number += 1
        
        if page_number == 5:
            break


    if job_list_data:
        df = pd.DataFrame(job_list_data)

        # Xóa các dòng trùng lặp
        df_unique = df.drop_duplicates()

        ## Save to .csv file
        #file_name = f"{SEARCH_KEYWORD.replace(' ', '_')}_all_jobs.csv"
        #df_unique.to_csv(file_name, index=False, sep = ';', encoding='utf-8-sig')

        ## Save to .xlsx file
        df_unique.to_excel('jobs_data.xlsx', index=False)
        print(f"Đã lưu {len(job_list_data)} job vào file 'jobs_data.xlsx' thành công!")
    else:
        print("Không có dữ liệu để lưu.")

In [7]:
if __name__ == "__main__":
    main()

Searching
--- Đang lấy dữ liệu từ trang 1 ---
Senior Data Engineer (AI/ML Analytics)
Reforged Labs (YC W22)
Ho Chi Minh City, Vietnam (Hybrid)
[{'Title': 'Senior Data Engineer (AI/ML Analytics)', 'Company': 'Reforged Labs (YC W22)', 'Location': 'Ho Chi Minh City, Vietnam (Hybrid)', 'Description': "About the job\nTop 3 reasons to join us\nBuild AI that powers top global gaming studios\nWorld-class team from Harvard & FAANG\nHigh salary and bonus with proven market fit\n\nAbout Us\nReforged Labs is a YC-backed, Sandhill Road funded AI startup revolutionizing how the world's top mobile gaming studios market themselves to hundreds of millions of players. We're building cutting-edge AI technology that powers sophisticated ad analysis and generation pipelines for elite global gaming companies.\n\nWe don't follow trends; we set them. Our data infrastructure supports technology that generates millions of dollars in value for our customers, processing billions of gameplay and ad-performance sig